In [8]:
%gui qt

import numpy as np

from plottr.data import datadict as dd
from plottr.node import node
from plottr.apps.tools import make_sequential_flowchart, make_sequential_flowchart_with_gui

In [23]:
class AutoNodeGui(node.NodeWidget):
    pass


class AutoNodeTemplate(node.Node):
    
    def addOption(self, name, specs):
        varname = '_'+name
        setattr(self, varname, specs.get("initialValue", None))
        
        def getter(self):
            return getattr(self, varname)
        
        @node.Node.updateOption(name)
        def setter(self, val):
            setattr(self, varname, val)
        
        setattr(self.__class__, name, property(getter, setter))
    
    def __init__(self, name):
        super().__init__(name)
        

def autoNode(nodeName, **options):
    
    def decorator(func):
        
        class AutoNode(AutoNodeTemplate):
            def __init__(self, name):
                super().__init__(name)
                for optName, optSpecs in options.items():
                    self.addOption(optName, optSpecs)
        
        AutoNode.__name__ = nodeName
        AutoNode.nodeName = nodeName
        AutoNode.autoNodeOptions = options
        AutoNode.process = func
        
        return AutoNode
    
    return decorator


@autoNode('MultiplierNode',
    multiplicationFactor={'initialValue' : 5, 'type' : int}
)
def multiply(self, **data):
    data = data['dataIn']
    print('process in', self.__class__)
    ret = data * self.multiplicationFactor
    print('about to return:', ret)
    return dict(dataOut=ret)

In [27]:
nodes, fc = make_sequential_flowchart([multiply])
multiplicationNode = nodes[0]
fc.setInput(dataIn=np.arange(3))

process in <class '__main__.autoNode.<locals>.decorator.<locals>.AutoNode'>
about to return: [ 0  5 10]


In [29]:
multiplicationNode.multiplicationFactor = 10

process in <class '__main__.autoNode.<locals>.decorator.<locals>.AutoNode'>
about to return: [ 0 10 20]


In [31]:
fc.output()

{'dataOut': array([ 0, 10, 20])}

process in <class '__main__.autoNode.<locals>.decorator.<locals>.AutoNode'>
about to return: [ 0  5 10]


{'dataOut': array([ 0, 10, 20])}

In [17]:
multiplicationNode.inputValues()

{'dataIn': array([0, 1, 2])}

In [18]:
multiplicationNode.outputValues()

{'dataOut': array([ 0,  5, 10])}

In [21]:
multiplicationNode.update()

process in <class '__main__.autoNode.<locals>.decorator.<locals>.AutoNode'>
about to return: [0 1 2]


In [38]:
issubclass(multiply, AutoNodeTemplate)

True